In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357289 sha256=c6edd13b926f81b63bcf7f42a1c5aa6d5a4a996fb3d289ea4a6c2fbb59766b54
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import pandas as pd
from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
from sklearn.metrics import average_precision_score

**Carga y Exploración de Datos**

In [ ]:
try:
    metadata = pd.read_csv('metadata.csv', usecols=['asset_id', 'content_id', 'end_vod_date'])
    train = pd.read_csv('train.csv', usecols=['account_id', 'asset_id', 'tunein'])
except FileNotFoundError as e:
    print(f"Error al cargar los archivos CSV: {e}")
    raise
except pd.errors.EmptyDataError as e:
    print(f"El archivo CSV está vacío: {e}")
    raise
except Exception as e:
    print(f"Ocurrió un error inesperado al cargar los datos: {e}")
    raise

In [ ]:
# Verificación de columnas y muestra de datos
print("Metadata:")
print(metadata.head())
print(metadata.columns)

print("\nTrain:")
print(train.head())
print(train.columns)

In [ ]:
# Conversión de fechas a formato datetime
try:
    metadata['end_vod_date'] = pd.to_datetime(metadata['end_vod_date'], errors='coerce')
    train['tunein'] = pd.to_datetime(train['tunein'], errors='coerce')
except Exception as e:
    print(f"Error al convertir las fechas: {e}")
    raise

**División del Dataset en Train y Test**

In [ ]:
try:
    train_df = train[train['tunein'] < '2021-03-01']
    test_df = train[train['tunein'] >= '2021-03-01']
except KeyError as e:
    print(f"Error en la división de los datos: {e}")
    raise

**Generación de Ratings Binarios**

In [ ]:
# Generación de ratings binarios
train_df['rating'] = 1

In [ ]:
# Filtramos contenidos disponibles después del 1 de marzo de 2021
try:
    available_content = metadata[metadata['end_vod_date'] >= '2021-03-01']['content_id'].unique()
except KeyError as e:
    print(f"Error al filtrar contenido disponible: {e}")
    raise

In [ ]:
# Verificación de valores nulos y conversión de tipos de datos
print("Valores nulos en 'asset_id' de train_df:", train_df['asset_id'].isnull().sum())
print("Valores nulos en 'asset_id' de metadata:", metadata['asset_id'].isnull().sum())

train_df = train_df[train_df['asset_id'].notnull()]
metadata = metadata[metadata['asset_id'].notnull()]

In [ ]:
try:
    train_df['asset_id'] = train_df['asset_id'].astype(int)
    metadata['asset_id'] = metadata['asset_id'].astype(int)
except ValueError as e:
    print(f"Error al convertir 'asset_id' a entero: {e}")
    raise

In [ ]:
# Unimos train_df con metadata para obtener 'content_id'
try:
    train_df = pd.merge(train_df, metadata[['asset_id', 'content_id']], on='asset_id', how='left')
except KeyError as e:
    print(f"Error al hacer el merge de los datos: {e}")
    raise

In [ ]:
# Limpieza de columnas duplicadas en train_df
train_df = train_df.loc[:, ~train_df.columns.duplicated()]

In [ ]:
# Verificación después del merge
print("\nColumnas en train_df después del merge:")
print(train_df.columns)
print(train_df.head())

In [ ]:
if 'content_id' not in train_df.columns:
    print("Error: La columna 'content_id' no se creó en el merge.")
else:
    print("\nTrain DataFrame después del merge:")
    print(train_df.head())

In [ ]:
# Filtramos por contenido disponible
try:
    train_df = train_df[train_df['content_id'].isin(available_content)]
except KeyError as e:
    print(f"Error al filtrar el DataFrame por contenido disponible: {e}")
    raise

In [ ]:
# Unimos test_df con metadata para obtener 'content_id'
try:
    test_df = pd.merge(test_df, metadata[['asset_id', 'content_id']], on='asset_id', how='left')
except KeyError as e:
    print(f"Error al hacer el merge del test set con metadata: {e}")
    raise

In [ ]:
# Limpieza de columnas duplicadas en test_df
test_df = test_df.loc[:, ~test_df.columns.duplicated()]

In [ ]:
# Verificación después del merge en test_df
print("\nColumnas en test_df después del merge:")
print(test_df.columns)
print(test_df.head())

**Factorización de Matrices (SVD)**

In [ ]:
# Verifica que 'content_id' esté presente en train_df
print("\nColumnas en train_df antes de cargar datos en Surprise:")
print(train_df.columns)

In [ ]:
try:
    reader = Reader(rating_scale=(1, 1))
    data = Dataset.load_from_df(train_df[['account_id', 'content_id', 'rating']], reader)
except KeyError as e:
    print(f"Error al cargar los datos en Surprise: {e}")
    raise

In [ ]:
# Dividimos los datos en train y test sets
try:
    trainset, testset = train_test_split(data, test_size=0.2)
except Exception as e:
    print(f"Error al dividir el dataset en train/test: {e}")
    raise

In [ ]:
# Entrenamos el modelo SVD
try:
    svd = SVD()
    svd.fit(trainset)
except Exception as e:
    print(f"Error al entrenar el modelo SVD: {e}")
    raise

In [ ]:
# Predecimos y evaluamos el modelo
try:
    predictions = svd.test(testset)
    accuracy.rmse(predictions)
except Exception as e:
    print(f"Error al realizar predicciones con SVD: {e}")
    raise

**Generación de Recomendaciones**

In [ ]:
def generar_recomendaciones_svd(model, account_id, train_df, num_recomendaciones=20):
    try:
        vistos = set(train_df[train_df['account_id'] == account_id]['content_id'].values)
        all_items = train_df['content_id'].unique()

        recomendaciones = [(item, model.predict(account_id, item).est) for item in all_items if item not in vistos]
        recomendaciones = sorted(recomendaciones, key=lambda x: x[1], reverse=True)[:num_recomendaciones]

        return [rec[0] for rec in recomendaciones]
    except KeyError as e:
        print(f"Error en la generación de recomendaciones: {e}")
        raise

In [ ]:
# Manejo de usuarios 'cold start'
def generar_recomendaciones_cold_start(train_df, num_recomendaciones=20):
    try:
        contenido_popular = train_df['content_id'].value_counts().index[:num_recomendaciones]
        return contenido_popular.tolist()
    except KeyError as e:
        print(f"Error en la generación de recomendaciones para cold start: {e}")
        raise

In [ ]:
# Ejemplo de recomendaciones para un usuario
try:
    account_id_ejemplo = test_df['account_id'].iloc[0]
    if account_id_ejemplo in train_df['account_id'].values:
        recomendaciones = generar_recomendaciones_svd(svd, account_id_ejemplo, train_df)
    else:
        print(f"Usuario {account_id_ejemplo} es un caso de cold start.")
        recomendaciones = generar_recomendaciones_cold_start(train_df)
    print(f"Recomendaciones para el usuario {account_id_ejemplo}: {recomendaciones}")
except IndexError as e:
    print(f"Error al intentar obtener el primer account_id de test_df: {e}")
    raise

In [ ]:
# Generación de recomendaciones para todos los usuarios en el test set
def verificar_recomendaciones(model, test_df, train_df, num_recomendaciones=20):
    recomendaciones_por_usuario = {}

    for account_id in test_df['account_id'].unique():
        if account_id in train_df['account_id'].values:
            recomendaciones = generar_recomendaciones_svd(model, account_id, train_df, num_recomendaciones)
        else:
            print(f"Usuario {account_id} es un caso de cold start.")
            recomendaciones = generar_recomendaciones_cold_start(train_df, num_recomendaciones)

        recomendaciones_por_usuario[account_id] = recomendaciones

    return recomendaciones_por_usuario

try:
    recomendaciones_por_usuario = verificar_recomendaciones(svd, test_df, train_df)
    for user, recs in recomendaciones_por_usuario.items():
        print(f"Recomendaciones para el usuario {user}: {recs}")
except Exception as e:
    print(f"Error al generar recomendaciones para todos los usuarios: {e}")

**Evaluación con MAP**

In [ ]:
def calcular_MAP_svd(model, test_df, train_df):
    scores = []
    try:
        for account_id in test_df['account_id'].unique():
            if account_id in train_df['account_id'].values:
                recomendaciones = generar_recomendaciones_svd(model, account_id, train_df)
            else:
                recomendaciones = generar_recomendaciones_cold_start(train_df)

            # Verificamos si las recomendaciones son parte del contenido visto por el usuario en el test set
            vistos = set(test_df[test_df['account_id'] == account_id]['content_id'].values)
            y_true = [1 if rec in vistos else 0 for rec in recomendaciones]
            y_score = [model.predict(account_id, rec).est for rec in recomendaciones]

            if len(vistos) > 0:
                scores.append(average_precision_score(y_true, y_score))
    except KeyError as e:
        print(f"Error al calcular MAP: {e}")
        raise

    return sum(scores) / len(scores) if scores else 0


In [ ]:
# Calcular y mostrar el MAP Score
try:
    map_score = calcular_MAP_svd(svd, test_df, train_df)
    print(f"MAP Score: {map_score}")
except Exception as e:
    print(f"Error al calcular el MAP Score: {e}")

**Conclusión**

En este proyecto, se desarrolló un sistema de recomendación empleando la factorización de matrices (SVD) sobre un conjunto de datos real de Telecom. El propósito fue generar 20 recomendaciones personalizadas para cada usuario, asegurando que no incluyeran contenido ya visualizado y que abarcaran también a aquellos usuarios sin historial previo (usuarios en cold start). La precisión del sistema se evaluó utilizando la métrica MAP, que permitió analizar la efectividad de las recomendaciones. Aunque el MAP Score obtenido fue bajo, esto refleja los desafíos inherentes a la generación de recomendaciones precisas en un escenario sin ratings explícitos y con restricciones en los datos. El proyecto pone de manifiesto la capacidad para aplicar técnicas avanzadas de recomendación y subraya la importancia de la optimización continua en sistemas de recomendación complejos.